In [95]:
print(f"ELK Upgrade Index Validation Logic -> Validate index count for Source/Dest ES cluster")

ELK Upgrade Index Validation Logic -> Validate index count for Source/Dest ES cluster


In [96]:
from elasticsearch import Elasticsearch
import os
import json

In [97]:
def get_headers():
    ''' Elasticsearch Header '''
    return {
            'Content-type': 'application/json', 
            'Authorization' : '{}'.format(os.getenv('BASIC_AUTH')),
            # 'Connection': 'close'
    }

In [98]:
def get_es_instance(host):
    es_client = Elasticsearch(hosts="http://{}".format(host), headers=get_headers(), timeout=5,  verify_certs=False)
    return es_client

es_obj_s_client = get_es_instance("localhost:9201")
es_obj_t_client = get_es_instance("localhost:9202")

In [99]:
def try_exists_index(es_client, index):
    try:
        if es_client.indices.exists(index):
            return True
        return False
            
    except Exception as e:
        # logging.error(e)
        print(e)
        pass

In [100]:
print(json.dumps(es_obj_s_client.cluster.health(), indent=2))

{
  "cluster_name": "supplychain-logging-dev",
  "status": "green",
  "timed_out": false,
  "number_of_nodes": 4,
  "number_of_data_nodes": 4,
  "active_primary_shards": 598,
  "active_shards": 1198,
  "relocating_shards": 0,
  "initializing_shards": 0,
  "unassigned_shards": 0,
  "delayed_unassigned_shards": 0,
  "number_of_pending_tasks": 0,
  "number_of_in_flight_fetch": 0,
  "task_max_waiting_in_queue_millis": 0,
  "active_shards_percent_as_number": 100.0
}


In [101]:
print(json.dumps(es_obj_t_client.cluster.health(), indent=2))

{
  "cluster_name": "docker-elasticsearch",
  "status": "yellow",
  "timed_out": false,
  "number_of_nodes": 1,
  "number_of_data_nodes": 1,
  "active_primary_shards": 74,
  "active_shards": 74,
  "relocating_shards": 0,
  "initializing_shards": 0,
  "unassigned_shards": 55,
  "delayed_unassigned_shards": 0,
  "number_of_pending_tasks": 0,
  "number_of_in_flight_fetch": 0,
  "task_max_waiting_in_queue_millis": 0,
  "active_shards_percent_as_number": 57.36434108527132
}


In [102]:
query = {
        # "_source": False,
	    'query': {
    	    'match_all': {}
        }
    }

In [103]:
''' extact a list of indices from the source cluster'''
source_idx_lists = es_obj_s_client.indices.get("*")
# logging.info(json.dumps(source_idx_lists, indent=2))
# logging.info(source_idx_lists)

In [104]:
def compare_docs_source_dest(es_client, es_t_client, source_idx_lists):
    is_not_exist_lists, different_doc, all_doc = [], [], []
    for each_index in source_idx_lists:
        ''' exclude system indices in the source cluster such as .monitoring-es-7-2024.07.12'''
        if '.' not in each_index:
            res_count_source, res_count_target = 0, 0
            ''' compare each index between source cluster and target cluster'''
            is_exist = try_exists_index(es_t_client, each_index)
            # logging.info(f"validate index [{each_index}] exsits : results is {is_exist}")
            ''' check the number of count'''
            res_count_source = es_client.count(index=each_index, body=query)["count"]
            if is_exist:
                res_count_target = es_t_client.count(index=each_index, body=query)["count"]
            
            if res_count_source != res_count_target:
                differ_dict = {
                                each_index : {
                                            "source_docs" : "%s" % res_count_source,
                                            "target_docs" : "%s" % res_count_target,
                                            "count" : "Differ"
                                            }
                               }
                
                different_doc.append(differ_dict)
                all_doc.append(differ_dict)
            else:
                 differ_dict = {
                                each_index : {
                                            "source_docs" : "%s" % res_count_source,
                                            "target_docs" : "%s" % res_count_target,
                                            "count" : "Same"
                                            }
                               }
                 all_doc.append(differ_dict)
                

            # print(res)
            if not is_exist:
                is_not_exist_lists.append(each_index)

    return all_doc, different_doc, is_not_exist_lists

In [105]:
all_doc, different_doc, is_not_exist_lists = compare_docs_source_dest(es_obj_s_client, es_obj_t_client, source_idx_lists)
# logging.info(different_doc)
# print("all_doc : {}".format(json.dumps(all_doc, indent=2)))

In [106]:
# print("different_doc : {}".format(json.dumps(different_doc, indent=2)))

In [107]:
# print("is_not_exist_lists : {}".format(json.dumps(is_not_exist_lists, indent=2)))